# Seminar Project: Distance Mapping Using Doc2Vec Embedding and PCA
---------------
``` 

Jakob Amann
Elias Gabriel Heppner, 1244414
Andri Rutschmann

### Loading Packages


In [6]:
import numpy as np 
import pandas as pd 
import time
import pickle
import time
import os
import datetime
from tqdm.auto import tqdm
import matplotlib.pyplot as plt


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\elias\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Load Speeches Data


In [7]:
#Define 
os.chdir("../data/opendiscourse")
os.getcwd()

FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: '../data/opendiscourse'

In [ ]:
import zipfile
import csv
import io

zip_file_path = 'sub_speeches.zip'

# Name of the CSV file within the zip file
csv_file_name = 'sub_speeches.csv'

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Read the CSV file directly into a Pandas DataFrame
    with zip_ref.open(csv_file_name) as file:
        speeches_df = pd.read_csv(file)


## Processing Functions

### Doc2Vec Encoding

In [ ]:

import gensim
import nltk
nltk.download('punkt')  # Nur notwendig beim ersten Mal
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models.phrases import Phrases, Phraser
import string
import nltk

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Define preprocessing function

def prep_doc2vec(speech_txts, df):
    
    def preprocess_speech(speech_txts, bigram=None, trigram=None):
        # Tokenization

        tokens = word_tokenize(speech_txts, language="german")
        
        # Lowercasing
        tokens = [token.lower() for token in tokens]
        
        # Removing punctuation
        tokens = [token for token in tokens if token not in string.punctuation]
        # Define parliamentary words
        #parl_words = ["kolleginnen", "kollegen", "damen", "herren", "kollege", "kollegin", "präsident", "präsidentin"]
        
        # Removing stopwords
        stop_words = set(stopwords.words('german'))
        tokens = [token for token in tokens if token not in stop_words]
        #tokens = [token for token in tokens if token not in parl_words]

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        # Apply bigram phrase model if available
        if bigram:
            tokens = bigram[tokens]
        
        # Apply trigram phrase model if available
        if trigram:
            tokens = trigram[tokens]

        return tokens


    bigram_phrases = Phrases(speech_txts, min_count=1)
    bigram = Phraser(bigram_phrases)
    trigram_phrases = Phrases(bigram[speech_txts], min_count=1)
    trigram = Phraser(trigram_phrases)

    df["tokenized"] = [preprocess_speech(doc, bigram=bigram, trigram=trigram) for doc in speech_txts]
    
    return df


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\elias\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elias\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elias\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id_x,session,electoral_term,date,document_url,speech_content,politician_id,first_name,last_name,faction_id,abbreviation,full_name,tokens,pol_name,tokenized
0,1053795,206,19,2021-01-28,https://dip21.bundestag.de/dip21/btp/19/19206.pdf,\n\nHerr Präsident! Liebe Kolleginnen und Koll...,11003608,Joachim,Pfeiffer,4,CDU/CSU,Christlich Demokratische Union Deutschlands/Ch...,"[herr, präsident, !, liebe, kolleginnen, und, ...",Joachim_Pfeiffer,"[herr, präsident, liebe, kolleginnen, kollegen..."
1,1050697,194,19,2020-11-25,https://dip21.bundestag.de/dip21/btp/19/19194.pdf,\n\nSehr geehrte Frau Präsidentin! Liebe Kolle...,11004659,Lisa,Badum,3,Grüne,Bündnis 90/Die Grünen,"[sehr, geehrte, frau, präsidentin, !, liebe, k...",Lisa_Badum,"[geehrte, frau, präsidentin, liebe, kolleginne..."
2,1052566,202,19,2020-12-17,https://dip21.bundestag.de/dip21/btp/19/19202.pdf,"\n\nLieber Kollege Whittaker, vielen Dank für ...",11004801,Sven,Lehmann,3,Grüne,Bündnis 90/Die Grünen,"[lieber, kollege, whittaker, ,, vielen, dank, ...",Sven_Lehmann,"[lieber, kollege, whittaker, vielen, dank, fra..."
3,1046545,180,19,2020-10-01,https://dip21.bundestag.de/dip21/btp/19/19180.pdf,\n\nLiebe Bürger! Frau Präsidentin! Meine Dame...,11004761,Leif Erik,Holm,0,AfD,Alternative für Deutschland,"[liebe, bürger, !, frau, präsidentin, !, meine...",Leif Erik_Holm,"[liebe, bürger, frau, präsidentin, damen, herr..."
4,1041775,164,19,2020-05-29,https://dip21.bundestag.de/dip21/btp/19/19164.pdf,\n\nSehr geehrter Herr Präsident! Liebe Kolleg...,11004004,Peter,Aumer,4,CDU/CSU,Christlich Demokratische Union Deutschlands/Ch...,"[sehr, geehrter, herr, präsident, !, liebe, ko...",Peter_Aumer,"[geehrter, herr, präsident, liebe, kolleginnen..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1039125,155,19,2020-04-22,https://dip21.bundestag.de/dip21/btp/19/19155.pdf,\n\nSehr geehrter Herr Präsident! Liebe Kolleg...,11004814,Astrid,Mannes,4,CDU/CSU,Christlich Demokratische Union Deutschlands/Ch...,"[sehr, geehrter, herr, präsident, !, liebe, ko...",Astrid_Mannes,"[geehrter, herr, präsident, liebe, kolleginnen..."
196,1058672,222,19,2021-04-16,https://dip21.bundestag.de/dip21/btp/19/19222.pdf,\n\nHerr Präsident! Liebe Kolleginnen und Koll...,11004270,Johannes,Fechner,23,SPD,Sozialdemokratische Partei Deutschlands,"[herr, präsident, !, liebe, kolleginnen, und, ...",Johannes_Fechner,"[herr, präsident, liebe, kolleginnen, kollegen..."
197,1040758,160,19,2020-05-14,https://dip21.bundestag.de/dip21/btp/19/19160.pdf,\n\nSehr geehrte Frau Präsidentin! Sehr geehrt...,11004791,Steffen,Kotré,0,AfD,Alternative für Deutschland,"[sehr, geehrte, frau, präsidentin, !, sehr, ge...",Steffen_Kotré,"[geehrte, frau, präsidentin, geehrte, damen, h..."
198,1055605,212,19,2021-02-25,https://dip21.bundestag.de/dip21/btp/19/19212.pdf,\n\nSehr geehrter Herr Präsident! Werte Kolleg...,11004719,Dietmar,Friedhoff,0,AfD,Alternative für Deutschland,"[sehr, geehrter, herr, präsident, !, werte, ko...",Dietmar_Friedhoff,"[geehrter, herr, präsident, werte, kolleginnen..."


In [ ]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

class DataFrameIterator(object):
    def __init__(self, dataframe, party_col=None, name_col=None, pos_col=None, gov_col=None):
        self.dataframe = dataframe
        self.party_col = party_col
        self.name_col = name_col
        self.pos_col = pos_col
        self.gov_col = gov_col

    def __iter__(self):
        for index, row in self.dataframe.iterrows():
            tags = []
            if self.party_col is not None:
                tags.append(str(row[self.party_col]))
            if self.name_col is not None:
                tags.append(str(row[self.name_col]))
            if self.pos_col is not None:
                tags.append(str(row[self.pos_col]))
            if self.gov_col is not None:
                tags.append(str(row[self.gov_col]))
            words = row['tokenized']
            yield TaggedDocument(words=words, tags=tags)

# Initialize Doc2Vec model
model = Doc2Vec(vector_size=200, window=20, min_count=50, workers=8, epochs=5)

# Tag documents using party and/or name as metadata
# If you only want to use polname, set party_column=None
# If you only want to use abbreviation, set name_column=None
# ...and so on
tagged_data = DataFrameIterator(filtered_df, party_col='abbreviation', name_col=None, pos_col=None, gov_col=None)

# Build vocabulary
model.build_vocab(tagged_data)

# Train the model
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)


### PCA

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


# Assuming you have already trained your Doc2Vec model and stored it in the variable `model`
def run_pca(df, title):
    
    # Infer vectors for each document in your corpus
    document_vectors = [model.infer_vector(doc) for doc in df["tokenized"]]

    # Convert the list of document vectors into a numpy array
    document_vectors_np = np.array(document_vectors)

    # Apply PCA
    pca = PCA(n_components=2)  # You can change the number of components as needed
    document_embeddings = pca.fit_transform(document_vectors_np)

    # Get the feature names (words) from the Doc2Vec model
    feature_names = model.wv.index_to_key

    # Create a dictionary to store word loadings
    word_loadings = dict(zip(feature_names, pc_loadings))

    # Sort the words based on their loadings
    sorted_words = sorted(word_loadings.items(), key=lambda x: abs(x[1]), reverse=True)

    # Get the top 10 words with highest loadings
    num_top_words = 10
    
    pc_loadings = pca.components_[0]
    pc_loads = pd.DataFrame(columns=['high', 'low'])

    # Get the top words with highest loadings
    top_words_highest_loadings = [word for word, _ in sorted_words[:num_top_words]]

    # Get the top words with lowest loadings
    top_words_lowest_loadings = [word for word, _ in sorted_words[-num_top_words:]]

    # Add the lists of top words to the DataFrame
    pc_loads['high'] = top_words_highest_loadings
    pc_loads['low'] = top_words_lowest_loadings
    pc_loads['pc'] = 1

    ##Plot

    # Define custom colors for each abbreviation
    abbreviation_colors = {
        'CDU/CSU': 'black',
        'DIE LINKE.': 'pink',
        'SPD': 'red',
        'AfD':'blue',
        'FDP': 'yellow',
        'Grüne': 'green',
        'not found': 'magenta',
    }

    # Plot the embeddings with color coding by string categories and custom colors
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(document_embeddings[:, 0], document_embeddings[:, 1], c=filtered_df["abbreviation"].map(abbreviation_colors), alpha=0.7)

    # Create custom legend
    legend_elements = [plt.Line2D([0], [0], marker='o', color=color, label=abbreviation, linestyle='') for abbreviation, color in abbreviation_colors.items()]
    legend1 = plt.legend(handles=legend_elements, title="Abbreviation", loc='upper right')

    plt.title(title)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.grid(True)

    # Add legend to the plot
    plt.gca().add_artist(legend1)

    plt.show()



Top 10 words with highest loadings:
geehrte: 0.18989089131355286
neuen: -0.18954887986183167
10: -0.1884608119726181
deutlich: 0.17944668233394623
euro: -0.1488945037126541
jahr: -0.13384661078453064
leider: 0.12915381789207458
sage: -0.12848737835884094
sehen: -0.1232459768652916
zwei: -0.12191121280193329

Top 10 words with lowest loadings:
dafür: 0.006304912734776735
„: 0.005805832799524069
seit: -0.005448547191917896
natürlich: 0.005315979477018118
fragen: -0.004989075940102339
regierung: 0.004953455179929733
großen: 0.003959269728511572
endlich: -0.0031430430244654417
krise: 0.001831629080697894
gilt: 0.0010227259481325746


## Application

### Topic 1: Covid

In [ ]:
#subselect for dataframe


#turn to list
speech_lst = speeches_df[speech_text].tolist()

